In [1]:
import datetime
import pandas as pd
import numpy as np
import collections
import re
from itertools import permutations

print (f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

2020-12-09 18:43:36


**--- Day 7: Handy Haversacks ---**

You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to issues in luggage processing.

Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

For example, consider the following rules:

```
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
```

These rules specify the required contents for 9 bag types. In this example, every faded blue bag is empty, every vibrant plum bag contains 11 bags (5 faded blue and 6 dotted black), and so on.

You have a shiny gold bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one shiny gold bag?)

In the above rules, the following options would be available to you:

```
    A bright white bag, which can hold your shiny gold bag directly.
    A muted yellow bag, which can hold your shiny gold bag directly, plus some other bags.
    A dark orange bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
    A light red bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
```

So, in this example, the number of bag colors that can eventually contain at least one shiny gold bag is 4.

How many bag colors can eventually contain at least one shiny gold bag? (The list of rules is quite long; make sure you get all of it.)



In [295]:
df = pd.read_csv('../data/data7.tsv', sep='\t', names=['i'])
df = df.replace(np.nan, '')
df['color_bag'] = df['i'].str.split('bag').str[0].str.strip()
df['bag_contains_1'] = df['i'].str.split('bag').str[1].str.split('contain ').str[1].str.strip()
df['bag_contains_2'] = df['i'].str.split('bag').str[2].str.split(', ').str[1].str.strip()
df['bag_contains_3'] = df['i'].str.split('bag').str[3].str.split(', ').str[1].str.strip()
df['bag_contains_4'] = df['i'].str.split('bag').str[4].str.split(', ').str[1].str.strip()

c_shiny_golds = df[df['i'].str.contains('shiny gold')]['color_bag'].values
cc_shiny_golds = df[df['i'].str.contains('|'.join(c_shiny_golds))]['color_bag'].values
ccc_shiny_golds = df[df['i'].str.contains('|'.join(cc_shiny_golds))]['color_bag'].values
cccc_shiny_golds = df[df['i'].str.contains('|'.join(ccc_shiny_golds))]['color_bag'].values
ccccc_shiny_golds = df[df['i'].str.contains('|'.join(cccc_shiny_golds))]['color_bag'].values
cccccc_shiny_golds = df[df['i'].str.contains('|'.join(ccccc_shiny_golds))]['color_bag'].values
ccccccc_shiny_golds = df[df['i'].str.contains('|'.join(cccccc_shiny_golds))]['color_bag'].values
cccccccc_shiny_golds = df[df['i'].str.contains('|'.join(ccccccc_shiny_golds))]['color_bag'].values
ccccccccc_shiny_golds = df[df['i'].str.contains('|'.join(cccccccc_shiny_golds))]['color_bag'].values
final_cc_shiny_golds = df[df['i'].str.contains('|'.join(ccccccccc_shiny_golds))]

In [242]:
final_cc_shiny_golds[~final_cc_shiny_golds['color_bag'].str.contains('shiny gold')].shape[0]

142

**--- Part Two ---**

It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!

Consider again your shiny gold bag and the rules from the above example:

```
    faded blue bags contain 0 other bags.
    dotted black bags contain 0 other bags.
    vibrant plum bags contain 11 other bags: 5 faded blue bags and 6 dotted black bags.
    dark olive bags contain 7 other bags: 3 faded blue bags and 4 dotted black bags.
```

So, a single shiny gold bag must contain 1 dark olive bag (and the 7 bags within it) plus 2 vibrant plum bags (and the 11 bags within each of those): 1 + 1*7 + 2 + 2*11 = 32 bags!

Of course, the actual rules have a small chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!

Here's another example:

```
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
```

In this example, a single shiny gold bag must contain 126 other bags.

How many individual bags are required inside your single shiny gold bag?

In [505]:
data = {}
with open("../data/data7.tsv", "r") as f:
    for line in f:
        words = line.split()
        description = " ".join(words[:2])
        tail = " ".join(words[4:])
        contents = []
        if tail != "no other bags.":
            for bag in tail.split(","):
                words = bag.strip().split(" ")
                contents.append((words[0], " ".join(words[1:3])))
        data[description] = contents
        
def luggage_content(maleta, data):
    rule = data[maleta]
    if rule == []:
        return []
    return [(n, color, luggage_content(color, data)) for n, color in rule]

def get_fold(n, l):
    total = 0
    if l != []:
        for n, _, c in l:
            total += int(n) * (1 + get_fold(n, c))
    return total

print(get_fold(1, luggage_content("shiny gold", data)))

10219
